In [1]:
import requests
import pandas as pd
import sqlite3

In [2]:
conn = sqlite3.connect('econ_data.db')

In [3]:
url = 'http://dataservices.imf.org/REST/SDMX_JSON.svc/'
param = [('name', 'Inflation'),
         ('dataset', 'IFS'),
         ('freq', 'M'),
         ('country', 'GB+DE+US'),
         ('series', 'PCPI_IX')]
series = '.'.join([param[x][1] for x in [2,3,4]])

key = 'CompactData/{}/{}'.format(param[1][1], series)

In [4]:
r = requests.get('{}{}'.format(url, key)).json()
df = pd.DataFrame()
for s in r['CompactData']['DataSet']['Series']:
    df[s['@REF_AREA']] = pd.Series([float(x['@OBS_VALUE']) for x in s['Obs']],
                                  index=[x['@TIME_PERIOD'] for x in s['Obs']])

df = df.pct_change(12).multiply(100).round(1)

C:\Users\BDew\Anaconda2\lib\site-packages\pandas\core\series.py:1295: RuntimeWarning: invalid value encountered in rint
  result = _values_from_object(self).round(decimals)


In [5]:
df.to_sql(param[0][1], conn, if_exists='replace', index_label='Date')

In [9]:
pd.read_sql("select * from Inflation", conn, index_col='Date', parse_dates=True).tail()

,GB,DE,US
Date,,,
2017-02,2.3,2.2,2.7
2017-03,2.3,1.6,2.4
2017-04,2.7,2.0,2.2
2017-05,2.9,1.5,1.9
2017-06,2.7,1.6,1.6
